# Todo
* Sensitivity 250 500 1000
* For the 500, take top 10% heightest weight
* Take that threshold for other topics


In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#import tempfile
#TEMP_FOLDER = tempfile.gettempdir()
#print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

#%matplotlib inline
#import matplotlib
#import matplotlib.pyplot as plt
#from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('retina')

from sklearn import preprocessing

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
from gensim.test.utils import datapath

### Read the pandas dataframe we got from data preparation

In [2]:
root_folder = Path('../Data/Pickle/')
file_name = 'TRACE2014_jinming'
file_path = root_folder / file_name

data = pd.read_pickle(file_path)

#### Group data by ['document','BOND_SYM_ID_transformed'] so that represent a document and token inside it

In [3]:
data_gb = data.groupby(by=['document','BOND_SYM_ID'])

In [4]:
matrix_1 = data_gb['BOND_SYM_ID'].size().unstack(fill_value=0)
matrix_1 = matrix_1.sort_index(axis=1)

matrix_1_shape = matrix_1.shape
print('We have {} documents and {} tokens'.format(matrix_1_shape[0],matrix_1_shape[1]))

We have 98304 documents and 12078 tokens


### Matrix 1 => Count of  BOND_SYM_ID in a document

#### Use SKlearn Label Encoder to transform tokens into integer

In [5]:
le = preprocessing.LabelEncoder()
le.fit(matrix_1.columns)
transform = le.transform(matrix_1.columns)
inverse_transform = le.inverse_transform(transform)

C:\Users\cchen07\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### Create id2word to map label encoding to its words then create corpus with gensim Dense2Corpus which takes numpy

In [10]:
# create id transformation and inverse_transformation dictionary as id2word for gensim 
id2word = dict(zip(transform, inverse_transform))
# load matrix as numpy arrays to gensim corpus
matrix1_corpus = gensim.matutils.Dense2Corpus(matrix_1.values,documents_columns=False)
# Run LdaMulticore in Gensim
num_topics = 1500
chunksize = 10000
passes = 20
iterations = 50
lda = gensim.models.ldamulticore.LdaMulticore(corpus= matrix1_corpus,id2word=id2word,workers=3, num_topics=num_topics, chunksize=chunksize, passes=passes,iterations=iterations)

print("LdaMulticore Done!!")

# The model name we want to save
model_name = "matrix1_{}topic".format(num_topics)
print("Saving Model as "+model_name)
# Get current dir
current_path = os.getcwd()
current_path = Path(current_path)
# Add model_name to 
save_path = current_path.parent / ("./Data/LDAModel/{}/".format(model_name))
# Create directory if not existed
try:
    os.mkdir(save_path)
except OSError:  
    print ("Creation of the directory %s failed" % save_path)
else:  
    print ("Successfully created the directory %s " % save_path)
# Concatonate save_path with model_name and load it as Gensim datapath
save_path = save_path / model_name
save_path = datapath(str(save_path))
# save the model
lda.save(save_path)
print("Model successfully save at" + save_path)

C:\Users\cchen07\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\models\ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


LdaMulticore Done!!
Saving Model as matrix1_1500topic
Successfully created the directory C:\Users\cchen07\Desktop\FINRA_TRACE-master\Data\LDAModel\matrix1_1500topic 
Model successfully save atC:\Users\cchen07\Desktop\FINRA_TRACE-master\Data\LDAModel\matrix1_1500topic\matrix1_1500topic
